## Descrição do Problema 
O problema do caixeiro viajante pode ser definido da seguinte forma: para uma determinada lista de bairros tendo as coordenadas exatas de latitude e longitude, podemos medir as distâncias entre cada par delas. Buscamos encontrar a rota mais curta possível que vai a cada bairro uma vez e retorna ao bairro de origem.

Existe uma classe de Problemas do Caixeiro Viajante que assume que a distância de ir do bairro $i$ para o bairro $j$ é a mesma que ir do bairro $j$ para o bairro $i$, este tipo de Problema do Caixeiro Viajante também é conhecido como o Problema do Caixeiro Viajante simétrico. Neste exemplo, usamos distâncias euclidianas, mas a formulação do modelo TSP é válida independentemente da forma como as distâncias individuais são determinadas.

## Solução Escolhida

A programação matemática é uma abordagem declarativa em que o modelador formula um modelo de otimização matemática que captura os principais aspectos de um problema de decisão complexo. O Gurobi Optimizer resolve esses modelos usando matemática e ciência da computação de última geração.

Um modelo de otimização matemática tem cinco componentes, os seguintes:

* Conjuntos e índices.
* Parâmetros.
* Variáveis ​​de decisão.
* Função(ões) objetiva(s).
* Restrições.

Respeitando esse modelo visto na aula de Pesquisa Operacional, apresentamos à seguir uma formulação que identifica a rota mais curta passando por todos os bairros/endereços apenas uma vez e retornando ao bairro de origem.

## Aplicando a fórmula 

### Conjuntos e índices
$i, j \in Bairros $: indices e conjuntos de endereço anexado por bairro.

$\text{Pairings}= \{(i,j) \in Bairros \times Bairros \}$: Conjunto de pares permitidos

$S \subset Bairros$: Um subconjunto do conjunto de bairros de Curitiba.

$G = (Bairros, Pairings)$: Um grafo onde o conjunto $Bairros$ define o conjunto de nós e o conjunto $Pairings$ define o conjunto de arestas. 

### Parâmetros

$d_{i, j} \in \mathbb{R}^+$: Distância do bairro $i$ ao bairro $j$, para todos os  $(i, j) \in Pares$. 

Observe que a distância do bairro $i$ ao bairro $j$ é a mesma que a distância do bairro $j$ ao bairro$i$, ou seja, $d_{i, j} = d_{j, i} $. Por esta razão, este problema do Caixeiro Viajante também é chamado de Problema do Caixeiro Viajante simétrico.

### Variavéis de decisão
$x_{i, j} \in \{0, 1\}$: Esta variável é igual a 1, se decidirmos conectar o bairro $i$ com o bairro $j$. Caso contrário, a variável de decisão é igual a zero.

### Função Objetivo
- **Rota mais curta**. Minimize a distância total de uma rota. Uma rota é uma sequência de bairros onde o entregador de pizza precisa visitar cada endereço apenas uma vez e retornar ao bairro inicial.

\begin{equation}
\text{Min} \quad Z = \sum_{(i,j) \in \text{Conjuntos}}d_{i,j} \cdot x_{i,j}
\tag{0}
\end{equation}

### Restrições
- **Restrições de simetria**. Para cada aresta $(i,j)$, certifique-se de que os bairros da cidade $i$ e $j$ estejam conectados, se o primeiro for visitado imediatamente antes ou depois da visita ao segundo.

\begin{equation}
x_{i, j} = x_{j, i} \quad \forall (i, j) \in Conjuntos
\tag{1}
\end{equation}

- **Entrada e saída de um bairro**. Para cada bairro $i$, certifique-se de que este endereço esteja conectado a dois outros endereços.

\begin{equation}
\sum_{(i,j) \in \text{Conjuntos}}x_{i,j} = 2 \quad \forall  i \in Bairros
\tag{2}
\end{equation}

- **Eliminação de Sub-rotas**. Essas restrições garantem que para qualquer subconjunto de endereços $S$ do conjunto de $Bairros$, não haja ciclo. Ou seja, não há uma rota que visite todas os endereços do subconjunto e retorne ao endereço de origem.

\begin{equation}
\sum_{(i \neq j) \in S}x_{i,j} \leq |S|-1 \quad \forall  S \subset  Bairros
\tag{3}
\end{equation}

- **Observação**. Em geral, se o número de bairros do problema é $n$, então o número possível de rotas é n\!.
Como há um número exponencial de restrições ($2^{n} - 2$) para eliminar ciclos, usamos restrições preguiçosas para eliminar esses ciclos dinamicamente.

In [19]:
# -- ORGANIZAÇÃO --
# Criar base de dados, no nosso caso bairros com as coordenadas de latitude e longitude do cliente que espera a pizza
# Calcular a distância entre todos os endereços marcados, com a combinação de todas entregas que o motoboy precisa fazer
# "só" aplicar o modelo de resolução do caixeiro viajante (TSP) isso desconsiderando sub-grafos conforme anotado nas restrições
# mostrar graficamente a solução e o mapeamento dos dados feito 

In [20]:
# gurobipy: provides Gurobi algorithms to solve MIP models.

%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
# BASE DE DADOS 

# Clientes mapeados para esta entrega específica de 5 pizzas. (Endereços de Mc Donalds variados na cidade de Curitiba)
# Passando por todos os pontos apenas uma vez e retornando ao ponto de origem ao final.

bairros = ['Cidade Industrial', 'Merces', 'Taruma', 'Centro', 'Batel']
coordenadas = {'Cidade Industrial': (-25.451551, -49.361174), 
               'Merces': (-25.421260, -49.291859), 
               'Taruma': (-25.436057, -49.235399), 
               'Centro': (-25.430311, -49.269434), 
               'Batel': (-25.439615, -49.280065)}

In [22]:
# CÁLCULO DE DISTÂNCIA 

import math
from itertools import combinations

# Calcula a matriz de distâncias em pares

def distance(city1, city2):
    c1 = coordenadas[city1]
    c2 = coordenadas[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(bairros, 2)}

In [23]:
# Imprimindo a Matriz
dist

{('Cidade Industrial', 'Merces'): 0.0756446555018882,
 ('Cidade Industrial', 'Taruma'): 0.12672574584905477,
 ('Cidade Industrial', 'Centro'): 0.0941666883775798,
 ('Cidade Industrial', 'Batel'): 0.08198254678283436,
 ('Merces', 'Taruma'): 0.05836679543199361,
 ('Merces', 'Centro'): 0.024182663749062795,
 ('Merces', 'Batel'): 0.02181752646383251,
 ('Taruma', 'Centro'): 0.03451662991950023,
 ('Taruma', 'Batel'): 0.04480748732075853,
 ('Centro', 'Batel'): 0.014127369783511737}

In [24]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model()

# Variáveis: o bairro 'i' é adjacente ao bairro 'j' no passeio?
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# Direção simétrica: Copie o objeto
for i, j in vars.keys():
    vars[j, i] = vars[i, j]  # aresta na direção oposta

# Restrições: duas arestas incidentes em cada bairro
cons = m.addConstrs(vars.sum(c, '*') == 2 for c in bairros)

In [31]:
# Callback - usando restrições preguiçosas para eliminar sub-rotas ; sub-grafos

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # lista de arestas selecionadas na solução
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # encontra o ciclo mais curto na lista de arestas selecionada
        tour = subtour(selected)
        if len(tour) < len(bairros):
            # adiciona a eliminação da sub-rota constr. para cada par de bairros na sub-rota
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Dada uma tuplelist de arestas, encontre a sub-rota mais curta

def subtour(edges):
    unvisited = bairros[:]
    cycle = bairros[:] # Dummy - substituição garantida
    while unvisited:  # true se a lista não estiver vazia
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # Nova sub-rota mais curta
    return cycle

In [32]:
# Relatório da busca feita
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 5 rows, 10 columns and 20 nonzeros
Model fingerprint: 0x5f146100
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolved: 5 rows, 10 columns, 20 nonzeros

Continuing optimization...


Explored 1 nodes (4 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 0.261134 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.611339832740e-01, best bound 2.611339832740e-01, gap 0.0000%

User-callback calls 24, time in user-callback 0.00 sec


In [33]:
# Prova real da solução

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(bairros)

In [34]:
# Representação gráfica da solução

import folium

map = folium.Map(location=[-25.441105, -49.276855], zoom_start = 12.2)

points = []
for bairro in tour:
  points.append(coordenadas[bairro])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map